We compare the macronutrional content of baby formula with that of cow's milk and then try to recreate the formula by blending milk with water and adding lactose.

TODO: background

# Data

As a reference for baby formula, we will use [Hipp PRE BIO COMBIOTIK](https://www.hipp.de/milchnahrung/produkte/hipp-combiotikR/hipp-bio-combiotikR-anfangsmilch/hipp-pre-bio-combiotikR/). They provide the following data on macronutrients:

    (Energy: 66 kcal / 100 ml)
    Fat: 3.5 g / 100 ml
    Carbs: 7.3 g / 100 ml
    Protein: 1.25 g / 100 ml

For cow's milk, we will use both whole milk as well as reduced-fat milk. The macronutrients as given for milk at LIDL are, for whole milk:

    (Energy: 68 kcal / 100 ml)
    Fat: 3.9 g / 100 ml
    Carbs: 4.9 g / 100 ml
    Protein: 3.4 g / 100 ml

And for reduced-fat milk:

    (Energy: 48 kcal / 100 ml)
    Fat: 1.6 g / 100 ml
    Carbs: 4.9 g / 100 ml
    Protein: 3.5 g / 100 ml


# Model

We setup an optimization model inspired by the classical [diet problem](https://en.wikipedia.org/wiki/Stigler_diet), where we want to use four ingredients (water, whole milk, reduced-fat milk, lactose) to match the nutrional profile of baby formula.

In [1]:
using JuMP
using GLPKMathProgInterface

In [2]:
m = Model(solver=GLPKSolverLP())

@variable(m, whole_milk ≥ 0)   # in 100 ml
@variable(m, lowfat_milk ≥ 0)  # in 100 ml
@variable(m, lactose ≥ 0)      # in g

@variable(m, slack_fat ≥ 0)
@variable(m, slack_carb ≥ 0)
@variable(m, slack_protein ≥ 0)

# fit liquid volume
@constraint(m, whole_milk + lowfat_milk ≤ 1)

# match fat content
@constraint(m, 3.9 * whole_milk + 1.6 * lowfat_milk == 3.5 + slack_fat)

# match carb content
@constraint(m, 4.9 * whole_milk + 4.9 * lowfat_milk + lactose == 7.3  + slack_carb )

# match protein content
@constraint(m, 3.4 * whole_milk + 3.5 * lowfat_milk == 1.25  + slack_protein)

# dummy objective
@objective(m, :Min, slack_carb + slack_fat + slack_protein);

In [3]:
status = solve(m)

:Optimal

In [4]:
@show getvalue(whole_milk), getvalue(lowfat_milk), getvalue(lactose)
@show getvalue(slack_fat), getvalue(slack_carb), getvalue(slack_protein)

(getvalue(whole_milk), getvalue(lowfat_milk), getvalue(lactose)) = (0.8974358974358976, 0.0, 2.902564102564101)
(getvalue(slack_fat), getvalue(slack_carb), getvalue(slack_protein)) = (0.0, 0.0, 1.8012820512820515)


(0.0, 0.0, 1.8012820512820515)

It seems that we can not get a perfect match. In particular, we will overshoot the protein content, or else not meet the fat content.

## Adding Canola Oil

In order to reduce the protein content, we need another non-dairy fat source. Let us use rapeseed oil which is often recommended for preparation of infant meals.

In [12]:
m = Model(solver=GLPKSolverLP())

@variable(m, whole_milk ≥ 0)   # in 100 ml
@variable(m, lowfat_milk ≥ 0)  # in 100 ml
@variable(m, lactose ≥ 0)      # in g
@variable(m, oil ≥ 0)  # in g

# fit liquid volume
@constraint(m, whole_milk + lowfat_milk ≤ 1)

# match fat content
@constraint(m, 3.9 * whole_milk + 1.6 * lowfat_milk + oil == 3.5)

# match carb content
@constraint(m, 4.9 * whole_milk + 4.9 * lowfat_milk + lactose == 7.3)

# match protein content
@constraint(m, 3.4 * whole_milk + 3.5 * lowfat_milk == 1.25)

# dummy objective
@objective(m, :Min, lactose + oil);

In [13]:
status = solve(m)

:Optimal

In [14]:
@show getvalue(whole_milk), getvalue(lowfat_milk), getvalue(lactose), getvalue(oil)

(getvalue(whole_milk), getvalue(lowfat_milk), getvalue(lactose), getvalue(oil)) = (0.36764705882352944, 0.0, 5.498529411764705, 2.0661764705882355)


(0.36764705882352944, 0.0, 5.498529411764705, 2.0661764705882355)

The interpretation of this solution is as follows: To reproduce 100 ml of baby formula, we should mix 37 ml of whole milk with 5.5 g of lactose, 2 ml of oil and about 60 ml of water.

Of course, this only takes